In [1]:
from models import Lenet
from datasets import MNIST
import torch
import torch.nn as nn
from pruning.unstructured import UnstructuredL1normPrune
# from pruning.unstructured import UnstructuredL1normPrune #has to write trainer class in diff file.
#has to import eval metrics 

c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\prabh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
lenet = Lenet()
mnist= MNIST()
train_dataloader, _ = mnist.get_dataloader()

In [3]:
criterion = criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.001)

In [4]:
# trainer = Trainer(lenet,5,train_dataloader,criterion,optimizer) (has to be implemented)
#loss =trainer.train()

In [5]:
trainer = UnstructuredL1normPrune(lenet,5,train_dataloader,criterion,optimizer,0.5)
trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 4.3451 | 
Epoch: 2 | train_loss: 3.0300 | 
Epoch: 3 | train_loss: 2.5501 | 
Epoch: 4 | train_loss: 2.1871 | 
Epoch: 5 | train_loss: 1.9063 | 
Training is done
Pruning is done


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.0944 | 
Epoch: 2 | train_loss: 1.7360 | 
Epoch: 3 | train_loss: 1.5149 | 
Epoch: 4 | train_loss: 1.3351 | 
Epoch: 5 | train_loss: 1.1621 | 
Retraining after pruning is done


Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
# pruned_model = Train_and_prune_and_retrain(lenet,5,train_dataloader,criterion,optimizer,0.5) #prune 50% of the weights  

In [7]:
# new_pruned_model =pruned_model.train_and_prune_and_retrain()

In [ ]:
#evaluations start here.